## Mobile App for Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

They want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

In this project, we'll try to figure out the questions from the historical data of the national 6/49 lottery game in Canada. The data set has data for 3,665 drawings, dating from 1982 to 2018.

## Create required functions

In [1]:
# create two functions that calculates factorial and combinations
def factorial(n):
    result = 1
    for i in range(1,n+1):
        result *= i
    
    return result

print(factorial(5))

def combinations(n,k):
    return factorial(n) / (factorial(k)*factorial(n-k))

print(combinations(5,3))

120
10.0


## One ticket probability
For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [2]:
def one_ticket_probability(user_numbers):
    
    # the total number of combinations
    # There are 49 possible numbers
    # and six numbers are sampled without replacement
    n_combinations = combinations(49, 6)
    probability = 1 / n_combinations * 100
    
    print('''Your chances to win the big prize with the numbers {} are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(user_numbers,
                    probability, int(n_combinations)))

# test
one_ticket_probability([1,2,3,4,5,6])

Your chances to win the big prize with the numbers [1, 2, 3, 4, 5, 6] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


## Compare tickets against the historical data

Users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now. We'll need to explore the historical data coming from the Canada 6/49 lottery. 

In [3]:
import pandas as pd

history = pd.read_csv('649.csv')
history.shape

(3665, 11)

In [4]:
history.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [5]:
history.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


To enable users to compare their ticket against the historical lottery data. we need to create a new function.

The function should prints:

* the number of times the combination selected occurred in the Canada data set; and
* the probability of winning the big prize in the next drawing with that combination.

We need to note that, inside the app, the user inputs six different numbers from 1 to 49. Under the hood, the six numbers will come as a Python list and serve as an input to our function.

In [6]:
# write a function that takes as input a row of the lottery dataframe 
# and returns a set containing all the six winning numbers

def extract_numbers(row):
    
    row = row[4:10]
    row = set(row.values)
    return row

# extract all the winning numbers
winning_set = history.apply(extract_numbers, axis=1)
winning_set.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [7]:
def check_historical_occurence(user_numbers, history_set):

    user_set = set(user_numbers)
    occurrence = user_set == history_set
    n_occurrences = occurrence.sum()
    
    if n_occurrences == 0:
        
        print('''The combination {} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, user_numbers))
        
    else:
        print('''The number of times combination {} has occured in the past is {}.
Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, n_occurrences, user_numbers))

        
#test 
check_historical_occurence([6,7,15,20,45,48], winning_set)

The combination [6, 7, 15, 20, 45, 48] has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination [6, 7, 15, 20, 45, 48] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


## Multi-ticket probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning. Hence, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
* Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [8]:
def multi_ticket_probability(n_tickets):
    
    n_combinations = combinations(49,6)
    probability = (n_tickets / n_combinations) * 100
    
    if n_tickets == 1:
        print('''Your chances to win the big prize with one ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(probability, int(n_combinations)))
    
    else:
        combinations_simplified = round(n_combinations / n_tickets)   
        print('''Your chances to win the big prize with {:,} different tickets are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_tickets, probability,
                                                               combinations_simplified))


test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for num in test_inputs:
    multi_ticket_probability(num)

Your chances to win the big prize with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
Your chances to win the big prize with 10 different tickets are 0.000072%.
In other words, you have a 1 in 1,398,382 chances to win.
Your chances to win the big prize with 100 different tickets are 0.000715%.
In other words, you have a 1 in 139,838 chances to win.
Your chances to win the big prize with 10,000 different tickets are 0.071511%.
In other words, you have a 1 in 1,398 chances to win.
Your chances to win the big prize with 1,000,000 different tickets are 7.151124%.
In other words, you have a 1 in 14 chances to win.
Your chances to win the big prize with 6,991,908 different tickets are 50.000000%.
In other words, you have a 1 in 2 chances to win.
Your chances to win the big prize with 13,983,816 different tickets are 100.000000%.
In other words, you have a 1 in 1 chances to win.


## Probabilities for two, three, four, or five winning numbers

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

We need to create a function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

* the user inputs six different numbers from 1 to 49; and an integer between 2 and 5 that represents the number of winning numbers expected
* Our function prints information about the probability of having the inputted number of winning numbers.

In [9]:
def probability_less_6(n_winning_numbers):
    
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(43, 6 - n_winning_numbers)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    
    n_combinations_total = combinations(49, 6)    
    probability = successful_outcomes / n_combinations_total
    
    probability_percentage = probability * 100    
    combinations_simplified = round(n_combinations_total/successful_outcomes)    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_numbers, probability_percentage,
                                                               int(combinations_simplified)))


In [10]:
# test 
for n in {2,3,4,5}:
    probability_less_6(n)

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
